In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp
from statistics import stdev

# Import scenario data
linecut = 96
scenario1 = pd.read_csv('data/Scenario1.csv').iloc[:linecut]
scenario2 = pd.read_csv('data/Scenario2.csv').iloc[:linecut]
scenario3 = pd.read_csv('data/Scenario3.csv').iloc[:linecut]
scenario4 = pd.read_csv('data/Scenario4.csv').iloc[:linecut]
scenario5 = pd.read_csv('data/Scenario5.csv').iloc[:linecut]
scenarios_data = [scenario1, scenario2, scenario3, scenario4, scenario5]

# Cost parameters
wind_cost_per_mw = 250
base_load_cost_per_mw = 360
peak_load_cost_per_mw = 200
load_following_option_fee = 50
load_following_exercise_fee = 18
service_fee = 5

max_wind_capacity = 20
max_generator_capacity = 10
solar_capacity = 1

# Decision variables (shared across scenarios)
wind_capacity = cp.Variable()
baseload_capacity = cp.Variable()
peak_capacity = cp.Variable()
load_following_capacity = cp.Variable()

# Constraints for shared variables
constraints = [
    wind_capacity >= 0,
    baseload_capacity >= 0,
    peak_capacity >= 0,
    load_following_capacity >= 0,
    wind_capacity <= max_wind_capacity,
    baseload_capacity + peak_capacity + load_following_capacity <= max_generator_capacity
]

# Fixed cost for all scenarios
fixed_cost = (
    wind_capacity * wind_cost_per_mw +
    baseload_capacity * base_load_cost_per_mw +
    peak_capacity * peak_load_cost_per_mw +
    load_following_capacity * load_following_option_fee
)

# Iterate over scenarios to define unique variables and costs
scenario_costs = []
i=0
real_time_cum = 0
for scenario in scenarios_data:
    time_steps = len(scenario)
    # Scenario-specific parameters as CVXPY constants
    demand_mw = cp.Parameter(time_steps, value=scenario["Demand [kW]"].values / 1000)
    real_time_price = cp.Parameter(time_steps, value=scenario["Real Time Price [$/MWh]"].values)
    wind_factor = cp.Parameter(time_steps, value=scenario["Wind Power Factor [p.u.]"].values)
    solar_factor = cp.Parameter(time_steps, value=scenario["Solar Power Factor [p.u.]"].values)

    # Renewable and base generation
    solar_gen = solar_capacity * solar_factor
    wind_gen = wind_capacity * wind_factor
    base_gen = baseload_capacity
    
    # Define peak gen using mask 1 for periods between 8 AM and 6 PM
    peak_active_mask = np.zeros(time_steps)
    peak_active_mask[8*4:18*4] = 1 
    peak_gen = peak_capacity * peak_active_mask

    # Calculate unmet demand
    unmet_demand = cp.pos(demand_mw - (solar_gen + wind_gen + base_gen + peak_gen))

    # Scenario-specific real-time variables
    load_follow_gen = cp.Variable(len(scenario), nonneg=True, name=f"load_follow_gen_{i}")
    spot_load = cp.Variable(len(scenario), nonneg=True, name=f"spot_load_{i}")

    # Real-time cost (load-following + spot market throughout day)
    real_time_cost = cp.sum(
        load_follow_gen * load_following_exercise_fee +  # Load-following cost
        spot_load * (real_time_price + service_fee)  # Spot market cost
    ) * 0.25

    # Real-time constraints for this scenario
    scenario_constraints = [
        load_follow_gen <= load_following_capacity,
        load_follow_gen + spot_load >= unmet_demand
    ]
    constraints.extend(scenario_constraints)

    # Total cost for this scenario
    real_time_cum += real_time_cost
    total_cost = fixed_cost + real_time_cost
    scenario_costs.append(total_cost)
    i+=1

print(constraints)

# Objective: Minimize the average cost across all scenarios
objective = cp.Minimize(sum(scenario_costs) / len(scenario_costs))

# Define and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)

# quick analysis
rt_avg = real_time_cum / len(scenario_costs) # avg cost to cover unmet demand
cost_sigma = stdev([cost.value for cost in scenario_costs]) # standard deviation of costs

# Output results
if problem.status == cp.OPTIMAL:
    output = {
        "Minimum expected cost": round(problem.value, 4),
        "realtime cost avg": round(rt_avg.value, 4),
        "stdev of cost": round(cost_sigma, 4),
        "Wind capacity (MW)": round(wind_capacity.value.item(), 4),
        "Baseload capacity (MW)": round(baseload_capacity.value.item(), 4),
        "Peak load capacity (MW)": round(peak_capacity.value.item(), 4),
        "Load following capacity (MW)": round(load_following_capacity.value.item(), 4)
    }
else:
    output = {"error": problem.status}

output

[Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Variable((), var1)), Inequality(Expression(AFFINE, UNKNOWN, ())), Inequality(Variable((96,), load_follow_gen_0, nonneg=True)), Inequality(Expression(CONVEX, NONNEGATIVE, (96,))), Inequality(Variable((96,), load_follow_gen_1, nonneg=True)), Inequality(Expression(CONVEX, NONNEGATIVE, (96,))), Inequality(Variable((96,), load_follow_gen_2, nonneg=True)), Inequality(Expression(CONVEX, NONNEGATIVE, (96,))), Inequality(Variable((96,), load_follow_gen_3, nonneg=True)), Inequality(Expression(CONVEX, NONNEGATIVE, (96,))), Inequality(Variable((96,), load_follow_gen_4, nonneg=True)), Inequality(Expression(CONVEX, NONNEGATIVE, (96,)))]
Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-13


c:\Users\liams\anaconda3\Lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
c:\Users\liams\anaconda3\Lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)
c:\Users\liams\anaco

{'Minimum expected cost': 3311.2064,
 'realtime cost avg': 296.1522,
 'stdev of cost': 57.6171,
 'Wind capacity (MW)': 0.1089,
 'Baseload capacity (MW)': 7.7281,
 'Peak load capacity (MW)': 0.6142,
 'Load following capacity (MW)': 1.6578}